In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim import Adam, SGD
import torchvision
import torchvision.transforms as transforms

import sys, os, math
import argparse

In [2]:
lr=0.01
data='cifar10'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar10_vgg_dist.pth'
resume = False

In [3]:
if data == 'cifar10':
    nclass = 10
    img_width = 32
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
#         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.CIFAR10(root=root, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testset = torchvision.datasets.CIFAR10(root=root, train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
if model == 'vgg':
    from models.vgg_dist import VGG_Dist
    net = nn.DataParallel(VGG_Dist(nclass, img_width=img_width).cuda())
    
net

DataParallel(
  (module): VGG_Dist(
    (conv1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=1)
    )
    (conv2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=1)
    )
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=1)
    )
    (conv4): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, tra

In [5]:
if resume:
    print(f'==> Resuming from {model_out}')
    net.load_state_dict(torch.load(model_out))

In [6]:
cudnn.benchmark = True

In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
def train(epoch):
    print('Epoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        pred = torch.max(outputs, dim=1)[1]
        correct += torch.sum(pred.eq(targets)).item()
        total += targets.numel()
    print(f'[TRAIN] Acc: {100.*correct/total:.3f}')

In [9]:
def test(epoch):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print(f'[TEST] Acc: {100.*correct/total:.3f}')

    # Save checkpoint after each epoch
    torch.save(net.state_dict(), model_out)

In [10]:
if data == 'cifar10':
    epochs = [40, 30, 20, 10]

In [11]:
count = 0

In [12]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0
[TRAIN] Acc: 13.000
[TEST] Acc: 21.330
Epoch: 1
[TRAIN] Acc: 30.388
[TEST] Acc: 33.640
Epoch: 2
[TRAIN] Acc: 45.996
[TEST] Acc: 55.460
Epoch: 3
[TRAIN] Acc: 61.666
[TEST] Acc: 56.210
Epoch: 4
[TRAIN] Acc: 69.372
[TEST] Acc: 70.760
Epoch: 5
[TRAIN] Acc: 74.990
[TEST] Acc: 66.570
Epoch: 6
[TRAIN] Acc: 78.022
[TEST] Acc: 73.030
Epoch: 7
[TRAIN] Acc: 80.622
[TEST] Acc: 73.520
Epoch: 8
[TRAIN] Acc: 82.422
[TEST] Acc: 70.260
Epoch: 9
[TRAIN] Acc: 83.580
[TEST] Acc: 78.160
Epoch: 10
[TRAIN] Acc: 85.134
[TEST] Acc: 80.710
Epoch: 11
[TRAIN] Acc: 86.208
[TEST] Acc: 82.820
Epoch: 12
[TRAIN] Acc: 86.856
[TEST] Acc: 79.940
Epoch: 13
[TRAIN] Acc: 87.888
[TEST] Acc: 78.630
Epoch: 14
[TRAIN] Acc: 88.330
[TEST] Acc: 83.990
Epoch: 15
[TRAIN] Acc: 88.948
[TEST] Acc: 80.740
Epoch: 16
[TRAIN] Acc: 89.432
[TEST] Acc: 85.320
Epoch: 17
[TRAIN] Acc: 90.156
[TEST] Acc: 84.830
Epoch: 18
[TRAIN] Acc: 90.800
[TEST] Acc: 84.070
Epoch: 19
[TRAIN] Acc: 91.104
[TEST] Acc: 83.660
Epoch: 20
[TRAIN] Acc: 91.374
